<a href="https://colab.research.google.com/github/kgpark88/energy-bigdata-analysis/blob/main/spark_dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spark DataFrame


## Spark 설치

In [9]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://downloads.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz 
!tar xf spark-3.3.2-bin-hadoop3.tgz 
!pip install -q findspark

import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.2-bin-hadoop3"

findspark.init()
findspark.find()

--2023-04-24 05:53:22--  https://downloads.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
Resolving downloads.apache.org (downloads.apache.org)... 88.99.95.219, 135.181.214.104, 2a01:4f8:10a:201a::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|88.99.95.219|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299360284 (285M) [application/x-gzip]
Saving to: ‘spark-3.3.2-bin-hadoop3.tgz.2’

spark-3.3.2-bin-had 100%[===================>] 285.49M  24.9MB/s    in 12s     

2023-04-24 05:53:35 (23.3 MB/s) - ‘spark-3.3.2-bin-hadoop3.tgz.2’ saved [299360284/299360284]



'/content/spark-3.3.2-bin-hadoop3'

## Creating a DataFrame

First we need to start a SparkSession:

In [10]:
from pyspark.sql import SparkSession

Then start the SparkSession

In [11]:
# May take a little while on a local computer
spark = SparkSession.builder.appName("Basics").getOrCreate()


You will first need to get the data from a file (or connect to a large distributed file like HDFS, we'll talk about this later once we move to larger datasets on AWS EC2).

In [12]:
df = spark.read.json("/content/spark-3.3.2-bin-hadoop3/examples/src/main/resources/people.json")

#### Showing the data

In [13]:
# Note how data is missing!
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [14]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [15]:
df.columns

['age', 'name']

In [16]:
df.describe()

DataFrame[summary: string, age: string, name: string]

Some data types make it easier to infer schema (like tabular formats such as csv which we will show later). 

However you often have to set the schema yourself if you aren't dealing with a .read method that doesn't have inferSchema() built-in.

Spark has all the tools you need for this, it just requires a very specific structure:

In [17]:
from pyspark.sql.types import StructField,StringType,IntegerType,StructType

Next we need to create the list of Structure fields
    * :param name: string, name of the field.
    * :param dataType: :class:`DataType` of the field.
    * :param nullable: boolean, whether the field can be null (None) or not.

In [18]:
data_schema = [StructField("age", IntegerType(), True),StructField("name", StringType(), True)]

In [19]:
final_struc = StructType(fields=data_schema)

In [20]:
df = spark.read.json("/content/spark-3.3.2-bin-hadoop3/examples/src/main/resources/people.json", schema=final_struc)

In [21]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



### Grabbing the data

In [22]:
df['age']

Column<'age'>

In [23]:
type(df['age'])

pyspark.sql.column.Column

In [24]:
df.select('age')

DataFrame[age: int]

In [25]:
type(df.select('age'))

pyspark.sql.dataframe.DataFrame

In [26]:
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [27]:
# Returns list of Row objects
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

Multiple Columns:

In [28]:
df.select(['age','name'])

DataFrame[age: int, name: string]

In [29]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



### Creating new columns

In [30]:
# Adding a new column with a simple copy
df.withColumn('newage',df['age']).show()

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    30|
|  19| Justin|    19|
+----+-------+------+



In [31]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [ ]:
# Simple Rename
df.withColumnRenamed('age','supernewage').show()

More complicated operations to create new columns

In [32]:
df.withColumn('doubleage',df['age']*2).show()

+----+-------+---------+
| age|   name|doubleage|
+----+-------+---------+
|null|Michael|     null|
|  30|   Andy|       60|
|  19| Justin|       38|
+----+-------+---------+



In [33]:
df.withColumn('add_one_age',df['age']+1).show()

+----+-------+-----------+
| age|   name|add_one_age|
+----+-------+-----------+
|null|Michael|       null|
|  30|   Andy|         31|
|  19| Justin|         20|
+----+-------+-----------+



In [34]:
df.withColumn('half_age',df['age']/2).show()

+----+-------+--------+
| age|   name|half_age|
+----+-------+--------+
|null|Michael|    null|
|  30|   Andy|    15.0|
|  19| Justin|     9.5|
+----+-------+--------+



In [35]:
df.withColumn('half_age',df['age']/2)

DataFrame[age: int, name: string, half_age: double]

We'll discuss much more complicated operations later on!

### Using SQL

To use SQL queries directly with the dataframe, you will need to register it to a temporary view:

In [36]:
# Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")

In [37]:
sql_results = spark.sql("SELECT * FROM people")

In [38]:
sql_results

DataFrame[age: int, name: string]

In [39]:
sql_results.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [40]:
spark.sql("SELECT * FROM people WHERE age=30").show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

